In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
SEED_BASE = 235

N_SPLITS = 5
VAL_DAYS = 30
STEP_DAYS = 30
GAP_HOURS = 0

N_TRIALS = 50
N_SEEDS_ENSEMBLE = 3

FINAL_NUM_BOOST_ROUNDS = 20000
EARLY_STOP = 400

EPS = 1e-6

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def make_fixed_horizon_folds(df, n_splits=5, val_days=30, step_days=30, gap_hours=0, time_col="delivery_start"):
    t = pd.to_datetime(df[time_col])
    tmax = t.max()

    folds = []
    for k in range(n_splits):
        val_end = tmax - pd.Timedelta(days=step_days * k)
        val_start = val_end - pd.Timedelta(days=val_days)
        train_end = val_start - pd.Timedelta(hours=gap_hours)

        tr_idx = df.index[t <= train_end]
        va_idx = df.index[(t > train_end) & (t <= val_end)]

        if len(tr_idx) < 1000 or len(va_idx) < 200:
            continue

        folds.append((tr_idx.to_numpy(), va_idx.to_numpy()))

    folds = folds[::-1]
    return folds

def recency_weights(df, half_life_days=90, time_col="delivery_start"):
    t = pd.to_datetime(df[time_col])
    age_days = (t.max() - t).dt.total_seconds() / 86400.0
    w = np.exp(-age_days / float(half_life_days))
    return w.to_numpy(dtype=np.float32)

def preprocess_and_engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["delivery_start"] = pd.to_datetime(df["delivery_start"])
    df["delivery_end"] = pd.to_datetime(df["delivery_end"])

    df["market"] = df["market"].astype("category")
    df = df.sort_values(["market", "delivery_start"]).reset_index(drop=True)

    ds = df["delivery_start"]
    df["hour"] = ds.dt.hour.astype(np.int8)
    df["dayofweek"] = ds.dt.dayofweek.astype(np.int8)
    df["month"] = ds.dt.month.astype(np.int8)
    df["dayofyear"] = ds.dt.dayofyear.astype(np.int16)
    df["weekofyear"] = ds.dt.isocalendar().week.astype(np.int16)
    df["dayofmonth"] = ds.dt.day.astype(np.int8)

    df["is_weekend"] = (df["dayofweek"] >= 5).astype(np.int8)
    df["is_peak_hour"] = ((df["hour"] >= 8) & (df["hour"] <= 20)).astype(np.int8)

    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24.0)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24.0)
    df["dow_sin"] = np.sin(2 * np.pi * df["dayofweek"] / 7.0)
    df["dow_cos"] = np.cos(2 * np.pi * df["dayofweek"] / 7.0)
    df["doy_sin"] = np.sin(2 * np.pi * df["dayofyear"] / 365.0)
    df["doy_cos"] = np.cos(2 * np.pi * df["dayofyear"] / 365.0)
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12.0)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12.0)

    wd = df["wind_direction_80m"].astype(np.float32)
    wd_rad = np.deg2rad(wd)
    df["wind_dir80_sin"] = np.sin(wd_rad)
    df["wind_dir80_cos"] = np.cos(wd_rad)

    df["net_load_forecast"] = df["load_forecast"] - (df["solar_forecast"] + df["wind_forecast"])
    df["temp_deviation"] = df["apparent_temperature_2m"] - df["air_temperature_2m"]
    df["wind_shear_approx"] = df["wind_speed_80m"] - df["wind_speed_10m"]

    df["net_load_x_peak"] = df["net_load_forecast"] * df["is_peak_hour"]
    df["cooling_degree_proxy"] = np.maximum(df["air_temperature_2m"] - 18.0, 0)
    df["heating_degree_proxy"] = np.maximum(18.0 - df["air_temperature_2m"], 0)

    df["wind_kinetic_10m"] = df["wind_speed_10m"] ** 3
    df["wind_kinetic_80m"] = df["wind_speed_80m"] ** 3
    df["solar_temp_penalty"] = df["solar_forecast"] * np.maximum(df["air_temperature_2m"] - 25.0, 0)

    df["t_hours"] = ((df["delivery_start"] - df["delivery_start"].min()).dt.total_seconds() / 3600.0).astype(np.float32)

    g_nl = df.groupby("market")["net_load_forecast"]
    df["net_load_ramp"] = g_nl.diff()
    df["solar_ramp"] = df.groupby("market")["solar_forecast"].diff()
    df["wind_ramp"] = df.groupby("market")["wind_forecast"].diff()

    base_cols = [
        "load_forecast", "solar_forecast", "wind_forecast", "net_load_forecast",
        "air_temperature_2m", "cloud_cover_total", "surface_pressure"
    ]
    lags = [1, 2, 3, 6, 12, 24, 48, 168]
    rolls = [6, 24, 48, 168]

    for col in base_cols:
        gc = df.groupby("market")[col]

        for L in lags:
            df[f"{col}_lag{L}"] = gc.shift(L)

        df[f"{col}_delta24"] = df[col] - df[f"{col}_lag24"]
        df[f"{col}_delta168"] = df[col] - df[f"{col}_lag168"]

        for W in rolls:
            df[f"{col}_roll{W}_mean"] = gc.transform(lambda x: x.rolling(W, min_periods=1).mean())
            df[f"{col}_roll{W}_std"] = gc.transform(lambda x: x.rolling(W, min_periods=2).std(ddof=0))

        df[f"{col}_ewm24"] = gc.transform(lambda x: x.ewm(span=24, adjust=False).mean())
        df[f"{col}_ewm168"] = gc.transform(lambda x: x.ewm(span=168, adjust=False).mean())

    return df

print("Loading data...")
train = pd.read_csv("train.csv")
test = pd.read_csv("test_for_participants.csv")

train["is_test"] = 0
test["is_test"] = 1
test["target"] = np.nan

df = pd.concat([train, test], ignore_index=True)

print("Feature engineering...")
df = preprocess_and_engineer_features(df)

raw_inputs = [
    "global_horizontal_irradiance", "diffuse_horizontal_irradiance", "direct_normal_irradiance",
    "cloud_cover_total", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high",
    "precipitation_amount", "visibility", "air_temperature_2m", "apparent_temperature_2m",
    "dew_point_temperature_2m", "wet_bulb_temperature_2m", "surface_pressure", "freezing_level_height",
    "relative_humidity_2m", "convective_available_potential_energy", "lifted_index", "convective_inhibition",
    "wind_speed_80m", "wind_direction_80m", "wind_gust_speed_10m", "wind_speed_10m",
    "solar_forecast", "wind_forecast", "load_forecast"
]
raw_inputs = [c for c in raw_inputs if c in df.columns]
df[raw_inputs] = df.groupby("market")[raw_inputs].ffill()

for c in df.columns:
    if df[c].dtype == "float64":
        df[c] = df[c].astype("float32")

train_df = df[df["is_test"] == 0].copy()
test_df = df[df["is_test"] == 1].copy()

train_df = train_df.sort_values("delivery_start").reset_index(drop=True)
test_df = test_df.sort_values("id").reset_index(drop=True)

drop_cols = ["id", "target", "delivery_start", "delivery_end", "is_test"]
features = [c for c in train_df.columns if c not in drop_cols]

X = train_df[features]
y = train_df["target"].astype(np.float32)
X_test = test_df[features]

cat_features = ["market"]

folds = make_fixed_horizon_folds(
    train_df,
    n_splits=N_SPLITS,
    val_days=VAL_DAYS,
    step_days=STEP_DAYS,
    gap_hours=GAP_HOURS
)
assert len(folds) > 0, "No valid folds constructed — reduce VAL_DAYS/STEP_DAYS or check timestamps."

def objective(trial):
    params = {
        "objective": "rmse",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt"]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 31, 255),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 200),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 2.0),
        "max_bin": trial.suggest_int("max_bin", 128, 512),
        "cat_smooth": trial.suggest_float("cat_smooth", 1.0, 100.0, log=True),
        "cat_l2": trial.suggest_float("cat_l2", 1e-3, 10.0, log=True),
        "n_estimators": 20000,
        "random_state": SEED_BASE,
        "n_jobs": -1,
        "verbose": -1,
        "force_col_wise": True,
    }

    if params["boosting_type"] == "dart":
        params["drop_rate"] = trial.suggest_float("drop_rate", 0.05, 0.3)
        params["skip_drop"] = trial.suggest_float("skip_drop", 0.2, 0.8)

    half_life = trial.suggest_float("half_life_days", 20.0, 180.0)
    w_all = recency_weights(train_df, half_life_days=half_life)

    scores = []
    for tr_idx, va_idx in folds:
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y.iloc[tr_idx], y.iloc[va_idx]
        wtr, wva = w_all[tr_idx], w_all[va_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            Xtr, ytr,
            sample_weight=wtr,
            eval_set=[(Xva, yva)],
            eval_metric="rmse",
            eval_sample_weight=[wva],
            categorical_feature=cat_features,
            callbacks=[lgb.early_stopping(EARLY_STOP, verbose=False)],
        )
        pred = model.predict(Xva, num_iteration=model.best_iteration_)
        scores.append(rmse(yva, pred))

    return float(np.mean(scores))

print(f"Optuna tuning ({N_TRIALS} trials)...")
study = optuna.create_study(
    study_name="power_market_lgbm_v1",
    storage="sqlite:///optuna_history.db",
    load_if_exists=True,
    direction="minimize"
)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

print(f"\nBest CV RMSE: {study.best_value:.4f}")
best = dict(study.best_params)

best_half_life = float(best.pop("half_life_days"))

final_params = {
    "objective": "rmse",
    "n_estimators": FINAL_NUM_BOOST_ROUNDS,
    "random_state": SEED_BASE,
    "n_jobs": -1,
    "verbose": -1,
    "force_col_wise": True,
    **best
}

print("\nTraining final multi-seed ensemble...")
test_pred = np.zeros(len(X_test), dtype=np.float64)
oof = np.zeros(len(X), dtype=np.float64)

seeds = [SEED_BASE + i for i in range(N_SEEDS_ENSEMBLE)]
w_all = recency_weights(train_df, half_life_days=best_half_life)

for s_i, seed in enumerate(seeds, 1):
    fold_pred = np.zeros(len(X_test), dtype=np.float64)
    fold_scores = []

    params_seeded = dict(final_params)
    params_seeded["random_state"] = seed

    for f, (tr_idx, va_idx) in enumerate(folds, 1):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y.iloc[tr_idx], y.iloc[va_idx]
        wtr, wva = w_all[tr_idx], w_all[va_idx]

        model = lgb.LGBMRegressor(**params_seeded)
        model.fit(
            Xtr, ytr,
            sample_weight=wtr,
            eval_set=[(Xva, yva)],
            eval_metric="rmse",
            categorical_feature=cat_features,
            callbacks=[lgb.early_stopping(EARLY_STOP, verbose=False)],
        )

        va_pred = model.predict(Xva, num_iteration=model.best_iteration_)
        fold_scores.append(rmse(yva, va_pred))
        oof[va_idx] += va_pred / len(seeds)

        fold_pred += model.predict(X_test, num_iteration=model.best_iteration_) / len(folds)

    print(f"Seed {seed} | mean fold RMSE: {np.mean(fold_scores):.4f}")
    test_pred += fold_pred / len(seeds)

valid_mask = oof != 0.0
actual_oof_rmse = rmse(y.iloc[valid_mask], oof[valid_mask])
print(f"\nOOF RMSE (on validated horizons only): {actual_oof_rmse:.4f}")

predictions = pd.DataFrame({
    "id": pd.to_numeric(test_df["id"], errors="raise").astype("int64"),
    "target": np.asarray(test_pred, dtype=np.float32),
})

assert len(predictions) == len(test_df)
assert predictions["id"].notna().all()
assert predictions["id"].duplicated().sum() == 0
assert predictions["target"].notna().all()
assert np.isfinite(predictions["target"]).all()

predictions = predictions.sort_values("id").reset_index(drop=True)

predictions.to_csv("aey__trading_submission.csv", index=False)
print("Saved: aey__trading_submission.csv")